Algorytm Dinitza służy do znajdywania maksymalnego przepływu między ustalonymi wierzchołkami. 

Dla grafu (ważonego) i wierzchołków początkowego v i końcowego u:

maksymalna przepustowosc=0

dopóki istnieje ścieżka z v do u:
   
    1. przypisujemy wierzchołkom poziomy zgodnie z BFS
   
    2. wybieramy konkretną scieżke zgodnie z DFS
   
    3. sprawdzenie ktora krawedz ze ścieżki ma najmniejsza przepustowosc
   
    4. dodajemy tę przepustowość do maksymalnej  

    5. modyfikujemy graf (zmmiejszamy przepustowość którą wykorzystaliśmy i dodajemy krawędzie przeciwne)

    6. powtarzamy

In [116]:
from collections import deque
import numpy as np
import sys
import queue

In [ ]:
#przypisanie poziomów zgodnie z przeszukaniem wszerz:

def BFS_level(graph,v_start, v_end):
    exist=0                                     #czy istnieje ścieżka z v do t? jeśli tak to =1
    level = {v: -1 for v in graph}              # -1 oznacza, że wierzchołek nie został odwiedzony
    queue = deque()                             #tworzenie kolejki
    queue.append(v_start)                       #dodajemy do kolejki wierzchołek początkowy
    level[v_start] = 0

    while queue:                                        #dopóki kolejka nie jest pusta
        u = queue.popleft()                             #bierzemy pierwszy wierzchołek z lewej
        for v, capacity in graph[u]:                    #dla każdej pary (sasiad, przepustowosc) dla tego wierzcholka
            if capacity > 0 and level[v] == -1:         #jeśli przepustowość>0 i nie jest jeszcze odwiedzony
                level[v] = level[u] + 1                 #przypisujemy mu poziom o 1 większy niż wierzchołka poprzedniego
                queue.append(v)                         #dodajemy ten nowy wierzchołek do kolejki
    
    if level[v_end]>-1:                                 # jeśli t został odwiedzony to istnieje ścieżka v-t
        exist=1
    return level, exist
    


def find_path(graph, level, v_start, v_end):        #zwraca scieżkę 
    path = [v_start]                                
    def dfs(u):
        if u == v_end:
            return True                             # ścieżka dotarła do końca

        for v, capacity in graph[u]:
            if capacity > 0 and level[v] == level[u] + 1 and v not in path:
                path.append(v)
                if dfs(v):
                    return True
                path.pop()                          # cofamy się – ścieżka nie działa

        return False                                # nie znaleziono ścieżki z tego wierzchołka

    if dfs(v_start):
        return path
    else:
        return None                                   # brak ścieżki

path=find_path(graph, BFS_level(graph, 's', 't')[0], v_start, v_end)
#print(path)

def maksymalna_przepustowość(graph, v_start, v_end, d):
    #d=1 - wyswietlenie szczegółów
    max_flow=0
    while BFS_level(graph, v_start, v_end)[1]==1:  #dopóki istnieje ścieżka z v do t:
        level=BFS_level(graph, v_start, v_end)[0]  #przypisanie poziomów wierzchołkom

        if d==1:
            print("poziomy:", level)

        path=find_path(graph, level, v_start, v_end)
        path_flow=[]
        #znajdujemy przepustowość ścieżki
        for i in range(len(path) - 1):
            u = path[i]
            v = path[i + 1]
                                                            # znajdujemy przepustowość krawędzi (u, v)
            for neighbor, capacity in graph[u]:         
                if neighbor == v:
                    path_flow.append(capacity)
                    break                                   # przerywamy, jak znajdziemy odpowiednią krawędź

        minimum = min(path_flow)
        max_flow=max_flow+minimum
        if d==1:
            print("ścieżka:", path)
            print("przepustowość ścieżki:",minimum)
        #aktualizujemy przepustowość krawedzi grafu
        for i in range(len(path) - 1):
            u = path[i]
            v = path[i + 1]
            for index, (neighbor, capacity) in enumerate(graph[u]):
                if neighbor == v:
                    new_capacity = capacity - minimum
                    if new_capacity > 0:                            #usuwamy krawędzie 0
                        graph[u][index] = (neighbor, new_capacity)
                    else:
                        graph[u].pop(index)
                    break  

            found_reverse = False                           #Dodaj/aktualizuj krawędź odwrotną (v, u)
            for index, (neighbor, capacity) in enumerate(graph[v]):
                if neighbor == u:
                    graph[v][index] = (neighbor, capacity + minimum)
                    found_reverse = True
                    break
            if not found_reverse:
                graph[v].append((u, minimum))                           
        
        if d==1:
            print("aktualny graf:")
            for node, neighbors in graph.items():
                print(f"{node}: {neighbors}")
        
    print("maksymalna przepustowość:", max_flow)
            

In [158]:
graph = {
    's': [('b', 10), ('c', 10)],
    'b': [('c', 2), ('e', 8), ('d', 4)],
    'c': [('e', 9)],
    'e': [('d',6), ('t', 10)],
    'd':[('t',10)],
    't':[]
}

v_start='s'
v_end='t'

maksymalna_przepustowość(graph, v_start, v_end,1)

poziomy: {'s': 0, 'b': 1, 'c': 1, 'e': 2, 'd': 2, 't': 3}
ścieżka: ['s', 'b', 'e', 't']
przepustowość ścieżki: 8
aktualny graf:
s: [('b', 2), ('c', 10)]
b: [('c', 2), ('d', 4), ('s', 8)]
c: [('e', 9)]
e: [('d', 6), ('t', 2), ('b', 8)]
d: [('t', 10)]
t: [('e', 8)]
poziomy: {'s': 0, 'b': 1, 'c': 1, 'e': 2, 'd': 2, 't': 3}
ścieżka: ['s', 'b', 'd', 't']
przepustowość ścieżki: 2
aktualny graf:
s: [('c', 10)]
b: [('c', 2), ('d', 2), ('s', 10)]
c: [('e', 9)]
e: [('d', 6), ('t', 2), ('b', 8)]
d: [('t', 8), ('b', 2)]
t: [('e', 8), ('d', 2)]
poziomy: {'s': 0, 'b': 3, 'c': 1, 'e': 2, 'd': 3, 't': 3}
ścieżka: ['s', 'c', 'e', 't']
przepustowość ścieżki: 2
aktualny graf:
s: [('c', 8)]
b: [('c', 2), ('d', 2), ('s', 10)]
c: [('e', 7), ('s', 2)]
e: [('d', 6), ('b', 8), ('c', 2)]
d: [('t', 8), ('b', 2)]
t: [('e', 10), ('d', 2)]
poziomy: {'s': 0, 'b': 3, 'c': 1, 'e': 2, 'd': 3, 't': 4}
ścieżka: ['s', 'c', 'e', 'd', 't']
przepustowość ścieżki: 6
aktualny graf:
s: [('c', 2)]
b: [('c', 2), ('d', 2), ('s', 1

In [161]:
graph ={
 's': [('a', 10), ('b', 5), ('c', 15)],
 'a': [('d', 9), ('b', 1)],
 'b': [('e', 6), ('c', 0)],
 'c': [('f', 15)],
 'd': [('t', 10)],
 'e': [('d', 1), ('t', 10)],
 'f': [('e', 5), ('t', 10)],
 't': []
}

In [162]:
v_start='s'
v_end='t'

maksymalna_przepustowość(graph, v_start, v_end,1)

poziomy: {'s': 0, 'a': 1, 'b': 1, 'c': 1, 'd': 2, 'e': 2, 'f': 2, 't': 3}
ścieżka: ['s', 'a', 'd', 't']
przepustowość ścieżki: 9
aktualny graf:
s: [('a', 1), ('b', 5), ('c', 15)]
a: [('b', 1), ('s', 9)]
b: [('e', 6), ('c', 0)]
c: [('f', 15)]
d: [('t', 1), ('a', 9)]
e: [('d', 1), ('t', 10)]
f: [('e', 5), ('t', 10)]
t: [('d', 9)]
poziomy: {'s': 0, 'a': 1, 'b': 1, 'c': 1, 'd': 3, 'e': 2, 'f': 2, 't': 3}
ścieżka: ['s', 'b', 'e', 't']
przepustowość ścieżki: 5
aktualny graf:
s: [('a', 1), ('c', 15)]
a: [('b', 1), ('s', 9)]
b: [('e', 1), ('c', 0), ('s', 5)]
c: [('f', 15)]
d: [('t', 1), ('a', 9)]
e: [('d', 1), ('t', 5), ('b', 5)]
f: [('e', 5), ('t', 10)]
t: [('d', 9), ('e', 5)]
poziomy: {'s': 0, 'a': 1, 'b': 2, 'c': 1, 'd': 4, 'e': 3, 'f': 2, 't': 3}
ścieżka: ['s', 'c', 'f', 't']
przepustowość ścieżki: 10
aktualny graf:
s: [('a', 1), ('c', 5)]
a: [('b', 1), ('s', 9)]
b: [('e', 1), ('c', 0), ('s', 5)]
c: [('f', 5), ('s', 10)]
d: [('t', 1), ('a', 9)]
e: [('d', 1), ('t', 5), ('b', 5)]
f: [('e', 5